<a href="https://colab.research.google.com/github/Tyton25/tools/blob/main/Expenses_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import pandas as pd
import numpy as np
import re

## Functions

In [259]:
def refactor_cols(my_df):
  columns = list(my_df.columns)
  columns = [c.lower() for c in columns]
  columns = [c.replace(' ', '_') for c in columns]
  my_df.columns = columns
  my_df.drop(columns='status', inplace=True)
  debit_df["date"] = pd.to_datetime(debit_df["date"])

  return my_df

In [580]:
def sum_orig_desc_col(raw_df):
  phone_pattern = r'\d{3}-\d{3}-\d{4}'
  url_patt = r'.*\.com'
  seven11_patt = r'7.*eleven'
  # my_df = raw_df[["original_description", "amount"]]
  my_df = raw_df.drop(columns=['date', 'description','category'])
  my_df['original_description'] = my_df['original_description'].map(lambda x: x.lower())
  my_df['original_description'] = my_df['original_description'].map(lambda x: x.split('*')[0])
  my_df['original_description'] = my_df['original_description'].map(lambda x: x.split('#')[0])
  my_df['original_description'] = my_df['original_description'].map(lambda x: re.split(phone_pattern, x)[0])
  my_df['original_description'] = my_df['original_description'].map(lambda x: re.split(url_patt, x)[0])
  my_df['original_description'] = my_df['original_description'].apply(lambda x: re.search(seven11_patt, x).group(0) if re.search(seven11_patt, x) else x)
  my_df['original_description'] = my_df['original_description'].apply(lambda x: re.search(r'doordash', x).group(0) if re.search(r'doordash', x) else x)
  my_df = my_df.groupby('original_description').agg('sum')
  my_df.sort_values(by=['original_description'], inplace=True)
  my_df.reset_index(inplace=True)

  return my_df

In [597]:
def sum_cat_col(raw_df):
  cat_df = raw_df.drop(columns=['date', 'original_description','description'])
  cat_df['category'] = cat_df['category'].map(lambda x: x.lower())
  cat_df = cat_df.groupby('category').agg("sum")
  cat_df.reset_index(inplace=True)

  return cat_df

In [602]:
def sum_desc_col(raw_df):
  url_patt = r'.*\.com'
  desc_df = raw_df.drop(columns=["date", "original_description", "category"])
  desc_df['description'] = desc_df['description'].map(lambda x: x.lower())
  desc_df['description'] = desc_df['description'].map(lambda x: re.split(url_patt, x)[0])
  desc_df = desc_df.groupby('description').agg('sum')
  desc_df.sort_values(by=['description'], inplace=True)
  desc_df.reset_index(inplace=True)

  return desc_df

## Debit Card Analysis

In [603]:
debit_df = pd.read_csv("/content/debit_transactions_Jan-Aug_2023.csv")
debit_df

,Date,Description,Original Description,Category,Amount,Status
0,2023-08-10,CARIBOU COFFEE CO #1 081023,CARIBOU COFFEE CO #1 081023,Category Pending,-4.66,Pending
1,2023-08-10,KRISPY KREME #4110 081023,KRISPY KREME #4110 081023,Category Pending,-89.78,Pending
2,2023-08-10,CMSVEND*CV DENVER 081023,CMSVEND*CV DENVER 081023,Category Pending,-1.35,Pending
3,2023-08-10,CMSVEND*CV DENVER 081023,CMSVEND*CV DENVER 081023,Category Pending,-1.35,Pending
4,2023-08-10,PBC- DENVER CO3834 081023,PBC- DENVER CO3834 081023,Category Pending,-2.10,Pending
...,...,...,...,...,...,...
626,2023-01-03,Vasa Fitness,VASA FITNESS 800-748-4949 UT,Gym,-21.99,Posted
627,2023-01-03,Rokfin,ROKFIN 512-3876531 TX,Entertainment,-14.99,Posted
628,2023-01-03,YouTube Premium,GOOGLE *YouTubePremium G.CO/HELPPAY#CA,Entertainment,-12.97,Posted
629,2023-01-03,7-Eleven,SEI 38135 BROOMFIELD CO,Gas,-7.32,Posted


In [604]:
debit_df = refactor_cols(debit_df)
debit_df

,date,description,original_description,category,amount
0,2023-08-10,CARIBOU COFFEE CO #1 081023,CARIBOU COFFEE CO #1 081023,Category Pending,-4.66
1,2023-08-10,KRISPY KREME #4110 081023,KRISPY KREME #4110 081023,Category Pending,-89.78
2,2023-08-10,CMSVEND*CV DENVER 081023,CMSVEND*CV DENVER 081023,Category Pending,-1.35
3,2023-08-10,CMSVEND*CV DENVER 081023,CMSVEND*CV DENVER 081023,Category Pending,-1.35
4,2023-08-10,PBC- DENVER CO3834 081023,PBC- DENVER CO3834 081023,Category Pending,-2.10
...,...,...,...,...,...
626,2023-01-03,Vasa Fitness,VASA FITNESS 800-748-4949 UT,Gym,-21.99
627,2023-01-03,Rokfin,ROKFIN 512-3876531 TX,Entertainment,-14.99
628,2023-01-03,YouTube Premium,GOOGLE *YouTubePremium G.CO/HELPPAY#CA,Entertainment,-12.97
629,2023-01-03,7-Eleven,SEI 38135 BROOMFIELD CO,Gas,-7.32


In [605]:
debit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631 entries, 0 to 630
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  631 non-null    datetime64[ns]
 1   description           631 non-null    object        
 2   original_description  631 non-null    object        
 3   category              631 non-null    object        
 4   amount                631 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 24.8+ KB


In [606]:
debit_df["amount"].sum().round(2)

-6728.87

In [607]:
debit_df.category.unique()

array(['Category Pending', 'Fast Food', 'Utilities',
       'Electronics & Software', 'Service & Parts', 'Food & Dining',
       'Gas', 'Coffee Shops', 'Cash', 'Restaurants',
       'Newspapers & Magazines', 'Atm Fee', 'Interest Income',
       'Mortgage Payment', 'Hosting', 'Paycheck', 'Mobile Phone',
       'Movies & Dvds', 'Doctor', 'Music', 'Eyecare', 'Personal Care',
       'Auto & Transport', 'Television', 'Shopping', 'Entertainment',
       'Financial', 'Credit Card Payment', 'Sporting Goods',
       'Gifts & Donations', 'Health & Fitness', 'Education', 'Check',
       'Clothing', 'Business Services', 'Uncategorized', 'Bonus',
       'Home Improvement', 'Dentist', 'Transfer', 'Groceries', 'Books',
       'State Tax', 'Federal Tax', 'Laundry', 'Taxes', 'Parking', 'Gym'],
      dtype=object)

In [608]:
category_totals_df = sum_cat_col(debit_df)
category_totals_df

,category,amount
0,atm fee,20.95
1,auto & transport,-289.32
2,bonus,998.70
3,books,-147.55
4,business services,-47.00
5,cash,-1230.95
6,category pending,-7226.75
7,check,-3382.13
8,clothing,-2559.32
9,coffee shops,-258.31


In [609]:
debit_desc_totals_df = sum_desc_col(debit_df)
debit_desc_totals_df[:50]

,description,amount
0,,-620.88
1,7-eleven,-1254.98
2,aaa membership,-185.00
3,adobe,-30.79
4,amazon web services,-8.39
5,amc theatres,-39.39
6,apple,-2341.72
7,arby's,-10.91
8,arby's via doordash,-52.44
9,atm fee rebate,20.95


In [610]:
debit_desc_totals_df[50:100]

,description,amount
50,doordash,-895.58
51,ebay,-171.85
52,eurest dining,-279.77
53,expresstoll,-75.60
54,fandango,-38.16
55,fitness gallery,-94.90
56,fleischmann family,-164.50
57,foundation hyundai,-28.72
58,four sea group i payroll,77755.09
59,front range family eye greenwood vilco,-1530.16


In [611]:
debit_orig_desc_df = sum_orig_desc_col(debit_df)
debit_orig_desc_df[:50]

,original_description,amount
0,,-3005.68
1,203 take 5 car wash broomfield co,-319.92
2,224pepsiven9147678600 denver co,-101.25
3,7-eleven,-170.00
4,7eleven,-80.00
5,aaa acg mbr 006/dues-ar 3037538800 co,-185.00
6,adobe,-20.80
7,adobe,-9.99
8,amazon web services aws.amazon.cowa,-8.39
9,amc 0092 w.minster pro westminster co,-39.39


In [612]:
debit_orig_desc_df[100:]

,original_description,amount
100,spotify,-54.00
101,sq,-4.33
102,starbucks store 00252 greenwood vilco,-9.15
103,starbucks store 06508 estes park co,-5.29
104,starbucks store 13426 erie co,-6.29
105,starbucks store 47645 broomfield co,-135.78
106,starbucks store 54769 broomfield co,-51.26
107,subway 32315 broomfield co,-34.12
108,synchrony bank cc pymt,-605.78
109,target t-192810445 reed swestminster co,-228.08


## Credit Card Analysis

In [613]:
cc_df = pd.read_csv("/content/cc_transactions_Jan-Aug_2023.csv")
cc_df

,Date,Description,Original Description,Category,Amount,Status
0,2023-08-11,TARGET.COM,TARGET.COM,Category Pending,-97.15,Pending
1,2023-08-11,KING SOOPERS #5086,KING SOOPERS #5086,Category Pending,-153.14,Pending
2,2023-08-11,WALMART.COM 8009666546,WALMART.COM 8009666546,Category Pending,-245.42,Pending
3,2023-08-11,USAA CREDIT CARD PAYMENT,USAA CREDIT CARD PAYMENT,Category Pending,7127.51,Pending
4,2023-08-08,AMAZON.COM AMZN.COM/BI,AMAZON.COM AMZN.COM/BI,Category Pending,-107.34,Pending
...,...,...,...,...,...,...
495,2023-03-13,PetSmart,PETSMART # 1458 WESTMINSTER CO,Pet Food & Supplies,-44.32,Posted
496,2023-03-13,USAA Credit Card,USAA CREDIT CARD PAYMENT SAN ANTONIO TX,Transfer,3000.00,Posted
497,2023-03-12,King Soopers,KING SOOPERS #5086 303-439-0113 CO,Groceries,-183.20,Posted
498,2023-03-12,Prime Video,Prime Video*HG9V41WS0 888-802-3080 WA,Television,-5.40,Posted


In [614]:
cc_df = refactor_cols(cc_df)
cc_df

,date,description,original_description,category,amount
0,2023-08-11,TARGET.COM,TARGET.COM,Category Pending,-97.15
1,2023-08-11,KING SOOPERS #5086,KING SOOPERS #5086,Category Pending,-153.14
2,2023-08-11,WALMART.COM 8009666546,WALMART.COM 8009666546,Category Pending,-245.42
3,2023-08-11,USAA CREDIT CARD PAYMENT,USAA CREDIT CARD PAYMENT,Category Pending,7127.51
4,2023-08-08,AMAZON.COM AMZN.COM/BI,AMAZON.COM AMZN.COM/BI,Category Pending,-107.34
...,...,...,...,...,...
495,2023-03-13,PetSmart,PETSMART # 1458 WESTMINSTER CO,Pet Food & Supplies,-44.32
496,2023-03-13,USAA Credit Card,USAA CREDIT CARD PAYMENT SAN ANTONIO TX,Transfer,3000.00
497,2023-03-12,King Soopers,KING SOOPERS #5086 303-439-0113 CO,Groceries,-183.20
498,2023-03-12,Prime Video,Prime Video*HG9V41WS0 888-802-3080 WA,Television,-5.40


In [615]:
cc_orig_desc_df = sum_orig_desc_col(cc_df)
cc_orig_desc_df[:50]

,original_description,amount
0,,-5189.44
1,1594 villa italian kit 303-4691737 co,-89.30
2,able kids foundation 9702262253 co,-261.96
3,afw - e-store,-988.45
4,amc 0092 w.minster pro westminster co,-32.35
5,amc online 9640,-32.19
6,amzn mktp us,-2191.77
7,amznfreetime,-27.00
8,arvada pediatric assoc,-633.43
9,atgpay,-655.83


In [616]:
cc_cat_df = sum_cat_col(cc_df)
cc_cat_df

,category,amount
0,alcohol & bars,-63.54
1,auto & transport,-195.06
2,baby supplies,-545.71
3,books,-101.27
4,category pending,6524.46
5,clothing,-563.62
6,coffee shops,-32.10
7,dentist,-809.60
8,doctor,-4576.31
9,entertainment,-17.50


In [617]:
cc_desc_df = sum_desc_col(cc_df)
cc_desc_df[:50]

,description,amount
0,,-449.91
1,able kids foundation,-261.96
2,afw e store,-988.45
3,amazon,-2597.53
4,amazon kids+,-16.20
5,amc theatres,-64.54
6,amznfreetime,-10.80
7,arvada pediatric assoc,-633.43
8,atgpay greenway park,-655.83
9,banfield,-268.82


In [618]:
cc_desc_df[50:]

,description,amount
50,panda express,-14.47
51,party city,-223.48
52,pediatric speech therapy,-2358.00
53,petsmart,-515.05
54,phillips 66,-327.23
55,pizza hut,-510.81
56,pretty nails inc,-78.00
57,prime video,-230.53
58,raising cane's,-125.22
59,rocky mountain gifts,-54.31


In [619]:
cc_desc_df = sum_orig_desc_col(cc_df)
cc_desc_df[:50]

,original_description,amount
0,,-5189.44
1,1594 villa italian kit 303-4691737 co,-89.30
2,able kids foundation 9702262253 co,-261.96
3,afw - e-store,-988.45
4,amc 0092 w.minster pro westminster co,-32.35
5,amc online 9640,-32.19
6,amzn mktp us,-2191.77
7,amznfreetime,-27.00
8,arvada pediatric assoc,-633.43
9,atgpay,-655.83
